In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_dir_base = "/Users/hn/Documents/01_research_data/RangeLand/Data/"
Shannon_data_dir = data_dir_base + "Shannon_Data/"
Min_data_dir_base = data_dir_base + "Min_Data/"

reOrganized_dir = data_dir_base + "reOrganized/"
os.makedirs(reOrganized_dir, exist_ok=True)

## We just need sheet A (beef cows) from CATINV

In [3]:
# CATINV = pd.read_excel(io=param_dir + "CATINV.xlsx", sheet_name=0)
xl = pd.ExcelFile(Shannon_data_dir + "CATINV.xls")
EX_sheet_names = xl.sheet_names  # see all sheet names
EX_sheet_names = EX_sheet_names[1:]
EX_sheet_names

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']

In [4]:
ii = 0
sheet_name_ = EX_sheet_names[ii]

curr_sheet = pd.read_excel(io = Shannon_data_dir + "CATINV.xls", 
                           sheet_name = sheet_name_, 
                           header = 0, skiprows = 0)
curr_sheet_columns = list(curr_sheet.columns)
named_columns = curr_sheet_columns[0] # [x for x in curr_sheet_columns if not("Unnamed" in x)]
named_columns

'Beef Cows'

In [5]:
curr_sheet.columns = list(curr_sheet.iloc[1, ].astype(str))
curr_sheet = curr_sheet[2:].copy()
curr_sheet.rename({'nan': 'state'}, axis=1, inplace=True)
curr_sheet.rename(columns={x: x.replace('.0', '') for x in curr_sheet.columns[1:]}, inplace=True)
curr_sheet.reset_index(drop=True, inplace=True)
curr_sheet.loc[:, curr_sheet.columns[1]:curr_sheet.columns[-1]] = curr_sheet.loc[:, \
                                                curr_sheet.columns[1]:curr_sheet.columns[-1]]*1000

# Drop rows that are entirely NA
curr_sheet.dropna(axis=0, how = 'all', inplace = True)

# Drop rows where state is NA
curr_sheet.dropna(subset=['state'], inplace = True)
Beef_Cows_CATINV = curr_sheet.copy()
# Beef_Cows_CATINV.sort_values(by=["state"], inplace=True)
Beef_Cows_CATINV.tail(4)

,state,1920,1921,1922,1923,1924,1925,1926,1927,1928,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
47,WV,60000.0,60000.0,60000.0,59000.0,58000.0,43000.0,34000.0,30000.0,30000.0,...,195000.0,200000.0,191000.0,185000.0,201000.0,207000.0,203000.0,198000.0,199000.0,190000.0
48,WI,37000.0,37000.0,35000.0,33000.0,30000.0,24000.0,20000.0,20000.0,16000.0,...,275000.0,260000.0,250000.0,275000.0,270000.0,290000.0,285000.0,290000.0,310000.0,310000.0
49,WY,300000.0,300000.0,300000.0,300000.0,296000.0,286000.0,281000.0,282000.0,282000.0,...,714000.0,694000.0,694000.0,694000.0,704000.0,714000.0,714000.0,714000.0,724000.0,702000.0
51,US,12525000.0,12292000.0,12182000.0,11974000.0,11926000.0,11204000.0,10294000.0,9439000.0,8926000.0,...,30281900.0,29631300.0,28956400.0,29332100.0,30163800.0,31170700.0,31466200.0,31690700.0,31338700.0,31157600.0


In [6]:
print(sorted(Beef_Cows_CATINV.state.unique()))

['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'US', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']


In [7]:
Beef_Cows_CATINV.shape

(51, 103)

In [8]:
out_name = reOrganized_dir + "Shannon_Beef_Cows_fromCATINV.csv"
Beef_Cows_CATINV.to_csv(out_name, index = False)

In [9]:
years = list(Beef_Cows_CATINV.columns[1:])
num_years = len(years)

CATINV_df_tall = pd.DataFrame(data=None, index=range(num_years*len(Beef_Cows_CATINV.state.unique())), 
                              columns=["state", "year", "inventory"], 
                              dtype=None, copy=False)

idx_ = 0 
for a_state in Beef_Cows_CATINV.state.unique():
    curr = Beef_Cows_CATINV[Beef_Cows_CATINV.state == a_state]
    CATINV_df_tall.loc[idx_: idx_ + num_years - 1 , "inventory"] = curr[years].values[0]
    CATINV_df_tall.loc[idx_: idx_ + num_years - 1 , "state"] = a_state
    CATINV_df_tall.loc[idx_: idx_ + num_years - 1 , "year"] = years
    idx_ = idx_ + num_years

In [10]:
CATINV_df_tall.head(5)

,state,year,inventory
0,AL,1920,103000.0
1,AL,1921,106000.0
2,AL,1922,108000.0
3,AL,1923,111000.0
4,AL,1924,114000.0


In [11]:
CATINV_df_tall[CATINV_df_tall.state!="US"].tail(5)

,state,year,inventory
5095,WY,2017,714000.0
5096,WY,2018,714000.0
5097,WY,2019,714000.0
5098,WY,2020,724000.0
5099,WY,2021,702000.0


In [12]:
Beef_Cows_CATINV[Beef_Cows_CATINV.state=="WY"]

,state,1920,1921,1922,1923,1924,1925,1926,1927,1928,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
49,WY,300000.0,300000.0,300000.0,300000.0,296000.0,286000.0,281000.0,282000.0,282000.0,...,714000.0,694000.0,694000.0,694000.0,704000.0,714000.0,714000.0,714000.0,724000.0,702000.0


In [13]:
import sys
sys.path.append("/Users/hn/Documents/00_GitHub/Rangeland/Python_Codes/")
import rangeland_core as rc


county_id_name_fips = pd.read_csv(Min_data_dir_base + "county_id_name_fips.csv")
county_id_name_fips.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

county_id_name_fips.sort_values(by=["state", "county"], inplace=True)

county_id_name_fips = rc.correct_Mins_FIPS(df=county_id_name_fips, col_="county")
county_id_name_fips.rename(columns={"county": "county_fips"}, inplace=True)

county_id_name_fips["state_fip"] = county_id_name_fips.county_fips.str.slice(0, 2)

print (len(county_id_name_fips.state.unique()))

county_id_name_fips = county_id_name_fips.drop(columns=["county_name", "county_fips", "fips"])
county_id_name_fips.drop_duplicates(inplace=True)
county_id_name_fips.reset_index(drop=True, inplace=True)
county_id_name_fips.head(2)

53


,state,state_fip
0,AK,02
1,AL,01


In [14]:
CATINV_df_tall.head(2)

,state,year,inventory
0,AL,1920,103000.0
1,AL,1921,106000.0


In [15]:
CATINV_df_tall = pd.merge(CATINV_df_tall, county_id_name_fips, on = ["state"], how = "left")
CATINV_df_tall.head(2)

,state,year,inventory,state_fip
0,AL,1920,103000.0,01
1,AL,1921,106000.0,01


In [17]:
CATINV_df_tall.year = CATINV_df_tall.year.astype(int)

In [19]:
CATINV_df_tall.head(2)

,state,year,inventory,state_fip
0,AL,1920,103000.0,01
1,AL,1921,106000.0,01


In [18]:
from datetime import datetime
import pickle

filename = reOrganized_dir + "Shannon_Beef_Cows_fromCATINV_tall.sav"

export_ = {"CATINV_annual_tall": CATINV_df_tall, 
           "source_code" : "convertShannonData",
           "Author": "HN",
           "Date" : datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

pickle.dump(export_, open(filename, 'wb'))

### We just need sheet A (beef cows) from ```Annual Cattle Inventory by State.xlsx```

In [27]:
xl = pd.ExcelFile(Shannon_data_dir + "Annual Cattle Inventory by State.xls")
EX_sheet_names = xl.sheet_names  # see all sheet names
EX_sheet_names = EX_sheet_names[1:]
EX_sheet_names

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']

In [28]:
ii = 0
sheet_name_ = EX_sheet_names[ii]

curr_sheet = pd.read_excel(io = Shannon_data_dir + "Annual Cattle Inventory by State.xls", 
                           sheet_name = sheet_name_, 
                           header = 0, skiprows = 0)
curr_sheet_columns = list(curr_sheet.columns)
named_columns = curr_sheet_columns[0] # [x for x in curr_sheet_columns if not("Unnamed" in x)]

print (f"{named_columns=}")
curr_sheet.head(4)

named_columns='Beef Cows'


,Beef Cows,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106
0,1000 Head,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1920.0,1921.0,1922.0,1923.0,1924.0,1925.0,1926.0,1927.0,1928.0,...,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2024.0,2025.0
2,AL,103.0,106.0,108.0,111.0,114.0,104.0,100.0,96.0,94.0,...,673.0,693.0,714.0,705.0,696.0,697.0,672.0,678.0,NaN,NaN
3,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,4.7,6.1,6.8,7.7,7.3,7.8,7.8,NaN,NaN


In [29]:
curr_sheet.columns = list(curr_sheet.iloc[1, ].astype(str))
curr_sheet = curr_sheet[2:].copy()

curr_sheet.rename({'nan': 'state'}, axis=1, inplace=True)
curr_sheet.rename(columns={x: x.replace('.0', '') for x in curr_sheet.columns[1:]}, inplace=True)
curr_sheet.reset_index(drop=True, inplace=True)

curr_sheet.loc[:, curr_sheet.columns[1]:curr_sheet.columns[-1]] = curr_sheet.loc[:, \
                                                curr_sheet.columns[1]:curr_sheet.columns[-1]]*1000


# Drop rows that are entirely NA
curr_sheet.dropna(axis=0, how = 'all', inplace = True)

# Drop rows where state is NA
curr_sheet.dropna(subset=['state'], inplace = True)
Beef_Cows_annual = curr_sheet.copy()
Beef_Cows_annual.head(2)

,state,1920,1921,1922,1923,1924,1925,1926,1927,1928,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,AL,103000.0,106000.0,108000.0,111000.0,114000.0,104000.0,100000.0,96000.0,94000.0,...,673000.0,693000.0,714000.0,705000.0,696000.0,697000.0,672000.0,678000.0,NaN,NaN
1,AK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4000.0,4700.0,6100.0,6800.0,7700.0,7300.0,7800.0,7800.0,NaN,NaN


In [30]:
Beef_Cows_annual.tail(4)

,state,1920,1921,1922,1923,1924,1925,1926,1927,1928,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
47,WV,60000.0,60000.0,60000.0,59000.0,58000.0,43000.0,34000.0,30000.0,30000.0,...,201000.0,207000.0,203000.0,198000.0,199000.0,190000.0,188000.0,191000.0,NaN,NaN
48,WI,37000.0,37000.0,35000.0,33000.0,30000.0,24000.0,20000.0,20000.0,16000.0,...,270000.0,290000.0,285000.0,290000.0,310000.0,290000.0,295000.0,280000.0,NaN,NaN
49,WY,300000.0,300000.0,300000.0,300000.0,296000.0,286000.0,281000.0,282000.0,282000.0,...,704000.0,714000.0,714000.0,714000.0,724000.0,702000.0,681000.0,671000.0,NaN,NaN
51,US,12525000.0,12292000.0,12182000.0,11974000.0,11926000.0,11204000.0,10294000.0,9439000.0,8926000.0,...,30163800.0,31170700.0,31466200.0,31690700.0,31338700.0,30843600.0,29983100.0,28917900.0,NaN,NaN


In [ ]:
out_name = reOrganized_dir + "Beef_Cows_fromAnnualCattleInventorybyState.csv"
Beef_Cows_CATINV.to_csv(out_name, index = False)

In [ ]:
print (f"{Beef_Cows_CATINV.shape=}")
print (f"{Beef_Cows_annual.shape=}")

In [ ]:
Beef_Cows_CATINV.head(4)

In [ ]:
Beef_Cows_annual.head(4)

In [ ]:
Beef_Cows_annual.loc[:, "1920":"2020"].equals(Beef_Cows_CATINV.loc[:, "1920":"2020"])

In [ ]:
Beef_Cows_annual.loc[:, "2021"] - (Beef_Cows_CATINV.loc[:, "2021"])

## We just need sheet B (beef cows) from Weekly Regional Cow Slaughter

In [ ]:
file_ = "Weekly Regional Cow Slaughter.xls"
xl = pd.ExcelFile(Shannon_data_dir + file_)
EX_sheet_names = xl.sheet_names  # see all sheet names
EX_sheet_names = EX_sheet_names[1:]
EX_sheet_names

In [ ]:
ii = 0
sheet_name_ = EX_sheet_names[ii]

curr_sheet = pd.read_excel(io = Shannon_data_dir + file_, 
                           sheet_name = sheet_name_, 
                           header = 0, skiprows = 4)
curr_sheet_columns = list(curr_sheet.columns)
curr_sheet.head(7)

In [ ]:
curr_sheet.loc[1, ] = curr_sheet.loc[0, ] + curr_sheet.loc[1, ]
curr_sheet = curr_sheet.loc[1:, ].copy()
curr_sheet.reset_index(drop=True, inplace=True)
curr_sheet.drop(axis=1, index=1, inplace=True)
curr_sheet.reset_index(drop=True, inplace=True)

for a_col in list(curr_sheet.columns):
    if not ("Unnamed" in a_col):
        curr_index = list(curr_sheet.columns).index(a_col)
        new_part = a_col.replace(".1", "").replace("- ", "").replace(" ", "_").replace("(", "").replace(")", "")
        
        curr_sheet.iloc[0, curr_index]= new_part + "_" + curr_sheet.iloc[0, curr_index].replace(" ", "")
        curr_sheet.iloc[0, curr_index+1] = new_part + "_" + curr_sheet.iloc[0, curr_index+1].replace(" ", "")

curr_sheet.iloc[0, 0] = "date"
curr_sheet.iloc[0, 1] = "week"
curr_sheet.rename(columns=curr_sheet.iloc[0], inplace = True)

# Drop first row
curr_sheet.drop(axis=1, index=0, inplace=True)
curr_sheet.reset_index(drop=True, inplace=True)

curr_sheet.head(7)

In [ ]:
curr_sheet["Region_1_&_Region_2_beef"] = curr_sheet["Region_1_&_Region_2_Beef&dairy"] - \
                                                   curr_sheet["Region_1_&_Region_2_dairy"]

for ii in range(3,11):
    curr_sheet["Region_" + str(ii) + "_beef"] = curr_sheet["Region_" + str(ii) + "_Beef&dairy"] - \
                                                   curr_sheet["Region_" + str(ii) + "_dairy"]

In [ ]:
out_name = reOrganized_dir + "Beef_Cows_fromWeeklyRegionalCowSlaughter.csv"
curr_sheet.to_csv(out_name, index = False)